Instalacion de librerias

In [1]:
pip install numpy matplotlib imageio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Importacion de librerias

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio

Codigo

In [10]:
# Crear carpetas para almacenar los frames y el GIF (si no existen)
frames_dir = 'frames'
gif_dir = 'gif'
os.makedirs(frames_dir, exist_ok=True)
os.makedirs(gif_dir, exist_ok=True)

# Definir la figura base (un cuadrado) en coordenadas homogéneas (2 filas: X y Y)
figura = np.array([
    [0, 0],
    [1, 0],
    [1, 1],
    [0, 1],
    [0, 0]  # Cerrar el cuadrado volviendo al punto inicial
]).T  # Transponer para que quede 2xN (más práctico para transformaciones)

# Función para crear la matriz de traslación
def matriz_traslacion(dx, dy):
    return np.array([
        [1, 0, dx],
        [0, 1, dy],
        [0, 0,  1]
    ])

# Función para crear la matriz de rotación
def matriz_rotacion(theta):
    return np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta),  np.cos(theta), 0],
        [0,              0,             1]
    ])

# Función para crear la matriz de escalado
def matriz_escala(sx, sy):
    return np.array([
        [sx, 0,  0],
        [0,  sy, 0],
        [0,  0,  1]
    ])

# Función para convertir la figura a coordenadas homogéneas (agregar fila de unos)
def hacer_homogenea(fig):
    ones = np.ones((1, fig.shape[1]))
    return np.vstack([fig, ones])

# Inicializar lista para guardar los frames
frames = []
num_frames = 60  # Definir el número de frames del GIF

# Generar cada frame
for t in range(num_frames):
    # Calcular parámetros de transformación que varían con el tiempo
    angulo = 2 * np.pi * t / num_frames             # Rotación continua (0 a 360 grados)
    escala = 1 + 0.5 * np.sin(2 * np.pi * t / num_frames)  # Escalado oscilante
    dx = 2 * np.cos(2 * np.pi * t / num_frames)     # Movimiento horizontal en círculo
    dy = 2 * np.sin(2 * np.pi * t / num_frames)     # Movimiento vertical en círculo

    # Crear las matrices de transformación correspondientes
    T = matriz_traslacion(dx, dy)
    R = matriz_rotacion(angulo)
    S = matriz_escala(escala, escala)

    # Combinar las transformaciones (primero escalar, luego rotar, luego trasladar)
    transformacion_total = T @ R @ S

    # Aplicar la transformación a la figura
    figura_homogenea = hacer_homogenea(figura)
    figura_transformada = transformacion_total @ figura_homogenea

    # Crear una nueva figura para graficar
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.plot(figura_transformada[0, :], figura_transformada[1, :], 'bo-')  # Dibujar el cuadrado transformado
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_aspect('equal')  # Escalas iguales en ambos ejes
    ax.set_title(f'Frame {t}')

    # Mostrar la matriz de transformación dentro del gráfico
    matriz_texto = np.array2string(transformacion_total, precision=2, suppress_small=True)
    ax.text(
        0.05, 0.95,
        f'Matriz:\n{matriz_texto}',
        fontsize=8,
        transform=ax.transAxes,
        verticalalignment='top',
        family='monospace',
        bbox=dict(boxstyle="round,pad=0.3", facecolor="white", edgecolor="gray")
    )

    # Guardar el frame generado en la carpeta correspondiente
    frame_path = os.path.join(frames_dir, f'frame_{t}.png')
    plt.savefig(frame_path)
    plt.close()  # Cerrar la figura para liberar memoria

    # Leer el frame guardado y agregarlo a la lista de frames
    frames.append(imageio.imread(frame_path))

# Crear el GIF animado a partir de los frames guardados
gif_path = os.path.join(gif_dir, 'transformacion.gif')
imageio.mimsave(gif_path, frames, duration=0.05)  # 0.05s entre frames (~20 fps)

# Imprimir la ruta donde se guardó el GIF
print(f"GIF guardado en: {gif_path}")

C:\Users\juanc\AppData\Local\Temp\ipykernel_12756\2754808513.py:95: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(frame_path))


GIF guardado en: gif\transformacion.gif
